In [1]:
import pandas as pd
import numpy as np
import os
import requests

from utils.simple_utils import *
from utils.google_utils import *
from utils.langchain_utils import *
from utils.best_hashtags import *

from mongo.interface import *

from chroma_interface import *
from chroma_interface.reader import *
from chroma_interface.writer import *

from script import *

keys = list(open(".env", "r"))

for item in keys:
    variable, value = item.split("=")[0], "=".join(item.split("=")[1:])
    os.environ[variable] = value.replace("\n", "")

country_code = 'AE' # Canada.
country = COUNTRY_CODES_TO_NAMES[country_code]
company_description = """For over 45 years, AGMC has delivered automotive and services to its customers as the exclusive importer for BMW Group and Motorrad with 6 locations spread across Dubai, Sharjah and the Northern Emirates. Established in 1976, AGMC serves as one of the largest for the BMW Group in the Middle East. AGMC is passionately committed to the communities it serves and demonstrates the BMW Group’s belief in the region’s potential by supporting local activities and events including an exclusive partnership with Dubai Autodrome to bring unparalleled on-track BMW driving experiences to the UAE and partnering with grassroots organisations as part of its commitment to the growth of the nation.In addition to its commitment to the local community, AGMC has implemented a series of corporate social responsibility initiatives through partnerships with youth road safety programmes, and its long-standing relationship with Dubai Police, through which the innovative BMW ConnectedDrive features are used, aimed to improve road safety for future generations and the advancement of mobility solutions across the Emirates"""
collection_name = "current_events"
query = f"""List 10 important and interesting events related to any of these topics: {','.join(QUERY_TOPICS)}. {company_description}"""

chroma_reader, chroma_writer = get_reader_writer(
        host=os.environ["CHROMA_IP"],
        port=int(os.environ["CHROMA_PORT"]),
        openai_api_key=os.environ["OPENAI_API_KEY"],
        reader_collection_name="current_events"
    )

In [2]:
generate_current_events(
    company_description=company_description, 
    chroma_reader=chroma_reader, 
    topic_list=QUERY_TOPICS, 
    keywords_dict=QUERY_KEYWORDS_DICT, 
    country_code=country_code,
    country=country,
    temperature=0.5
)

getting holiday list
Calling https://api.api-ninjas.com/v1/holidays?country=AE&year=2023&type=
Calling https://api.api-ninjas.com/v1/holidays?country=AE&year=2024&type=
['National Day']
Search relevant documents


100%|██████████| 13/13 [00:06<00:00,  2.01it/s]
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


United Arab Emirates National Day||National Day||It is a significant patriotic event celebrated by the people of the UAE, with various cultural and traditional activities.
Dubai International Film Festival||Film Festival||One of the most prestigious film festivals in the Middle East, showcasing a wide range of international and regional films, attracting global attention.
Abu Dhabi Grand Prix||Formula 1 Race||One of the most popular and thrilling Formula 1 races held in the UAE, drawing in motorsport fans from around the world.
Dubai Shopping Festival||Shopping Festival||An internationally renowned shopping extravaganza offering amazing discounts, entertainment, and events, attracting visitors from various countries.
Dubai World Cup||Horse Racing||One of the richest horse races in the world, featuring top-class horses, jockeys, and entertainment, with a global audience.


[{'event_name': 'United Arab Emirates National Day',
  'validation': {'google_trends': ['United Arab Emirates National Day']},
  'topic': 'National Day'},
 {'event_name': 'Dubai International Film Festival',
  'validation': {'google_trends': ['Dubai International Film Festival']},
  'topic': 'Film Festival'},
 {'event_name': 'Abu Dhabi Grand Prix',
  'validation': {'google_trends': ['Abu Dhabi Grand Prix']},
  'topic': 'Formula 1 Race'},
 {'event_name': 'Dubai Shopping Festival',
  'validation': {'google_trends': ['Dubai Shopping Festival']},
  'topic': 'Shopping Festival'},
 {'event_name': 'Dubai World Cup',
  'validation': {'google_trends': ['Dubai World Cup']},
  'topic': 'Horse Racing'}]

In [ ]:
docs = []
for topic in tqdm(QUERY_TOPICS):

    keywords = QUERY_KEYWORDS_DICT[topic]
    query = f"{topic} important events in  country: {COUNTRY_CODES_TO_NAMES[country_code]}"

    docs.extend(chroma_reader.search(
        query=query,
        n=5,
        filter={"country_code":country_code},
        keywords=keywords
    ))

In [ ]:
docs

In [ ]:
query = f"{','.join(QUERY_TOPICS)} important events"
keywords = QUERY_KEYWORDS

docs = chroma_reader.search(
        query=query,
        n=30,
        filter={"country_code":country_code},
        keywords=keywords
    )


In [ ]:
output = generate_current_events(
    company_description=company_description,
    chroma_reader=chroma_reader,
    topic_list=QUERY_TOPICS,
    keywords=QUERY_KEYWORDS,
    country_code=country_code
)

In [ ]:
output1 = output

In [ ]:
output

In [ ]:
def generate_current_events(company_description:str, chroma_reader:Reader, topic_list:list[str], keywords:dict, country_code:str, llm_name:str="gpt-3") -> list[dict]:
    print("getting holiday list")
    holidays = str(get_holidays(country_code=country_code)["name"].to_list())
    print(holidays)

    print("Search relevant documents")

    relevant_docs = []
    for topic in tqdm(topic_list):

        keywords = keywords[topic]
        query = f"{topic} important events" + company_description

        relevant_docs.extend(chroma_reader.search(
            query=query,
            n=5,
            filter={"country_code":country_code},
            keywords=keywords
        ))

    relevant_docs_text = "\n".join([item.page_content.replace("\n", " ") for item in relevant_docs])

    template = """
I am a marketing head and I am tasked to write social media content about big events around this date: {date}
I am looking for important events to write some content (like Instagram post/reel, or Tiktok video, or Facebook post, or linkedin post, etc.).
There is no specific topic that I am interested in, I write about general events/ideas.
But I only write about events that have a massive reach within a country or worldwide (massive events in the specified is preferred).
For example christmas which is celebrated all around the world, or diwali which is one of the biggest festivals in India, or grammy awards is the biggest music award show in the US, or FIFA world cup which is watched all over the world, etc.
The above mentioned events are just examples that I would write content about.

Important details:
Country that I am writing about: {country_code} (This is in the ISO 3166 alpha 2 format)
Date: {date}, This is the date around which I want to write content.

List at least 5 events.

Important calendar events for country={country_code}: {holidays}

I searched the internet about upcoming events, here is some data that you can use to suggest me what events I should write about:
{context}

Give me output in the following format only: 
<event name>||<topic>||<short reason>
<event name>||<topic>||<short reason>
and so on

Instructions:
1. The order of events should represent how engaging the event might be.
2. Do not restrict the events to only one or two categories like sports or film, there should be multiple categories if possible.
3. List only those events that are currently going on, going to happen in 1 month, or have happened within 1 month of today's date.
4. Only give me events that having a positive sentiment.
5. Events should be important on a global or a national scale, example: sporting events like cricket tournament, international conferences like G20 or BRICS, religious festival like christmas or eid, etc.
6. Dont write numbering in the front
7. Keep the items diverse, and prioritise events that have a larger reach.
8. Do not repeat ideas.
"""

    prompt = PromptTemplate(template=template, input_variables=["context", "date", "country_code", "holidays"])
    chain = LLMChain(prompt=prompt, llm=get_model(llm_name, temperature=0.2), output_key="events")

    output = chain({
        "date": current_date(),
        "context": relevant_docs_text,
        "holidays": holidays,
        "country_code": country_code
    })["events"]

    output = re.sub("(\n)+", "\n", output)

    outputs = output.split("\n")
    outputs = [re.sub("^(\d+)\.?", "", k) for k in outputs]
    outputs = [re.sub("\s+\|\|\s+", "||", k) for k in outputs]

    sorted_headlines = [item for item in outputs if (len(item.split("||")) == 4) or (len(item.split("||")) == 3)]
    
    data = []

    for item in sorted_headlines:
        try:
            title, topic, reason = item.split("||")
        except:
            title, topic, sentiment = item.split("||")
            reason = ""

        data.append({
            "event_name": title,
            "validation": {
                "google_trends": [title]
            },
            "topic": topic
        })

    return data